In [2]:
import pandas as pd
from IPython.display import JSON
import pylab
import json
%matplotlib inline
import matplotlib.pyplot as plt
#plt.style.use('seaborn-whitegrid') # TODO: what did this do?
import numpy as np
from IPython.core import display
import psycopg2
import os
homedir = os.getenv("HOME")
with open(homedir + "/.pgpass",'r') as pgfile:
    pgdat = pgfile.read().strip().split(":")
    pgurl = pgdat[0]
    pgport = pgdat[1]
    pgdb = pgdat[2]
    pguser = pgdat[3]
    pgpass = pgdat[4]

%load_ext sql
from sqlalchemy import create_engine
# sql info: https://github.com/catherinedevlin/ipython-sql
# on error `pip install ipython-sql`
#uncomment for remote
pgport=3333
#pgport=5433
engine = create_engine('postgresql://postgres@localhost:%i/postgres' % pgport)
db = psycopg2.connect(host=pgurl, database=pgdb, user=pguser,password=pgpass,port=pgport)

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [3]:
def qry(q):
  return pd.read_sql_query(con = db, sql = q)
# pd.set_option("max_colwidth", None)
# pd.set_option('display.max_columns', None)
pd.set_option("display.latex.repr", False)
pd.set_option("display.latex.escape", True)

def iDisplay(v):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, "max_colwidth", None):
        display.display(v)


Count number of uploaded APK files

In [194]:
%%sql postgresql://postgres@localhost:$pgport/postgres
select
    (select count(*) from apks where apkname not like 'jar_%' and apkname not like 'spec_%') as apk_count,
    (select count(*) from apks) as blob_count, 
    (select count(*) from inputs) as input_count, 
    (select count(*) from jobs) as job_count, 
    (select count(*) from resultdata) as result_data_count, 
    (select count(*) from results) as result_count

1 rows affected.


apk_count,blob_count,input_count,job_count,result_data_count,result_count
55,57,1,1410,1336,1368


In [195]:
%%sql -- count locations by pattern
select count(*), cast(jobs.config::json->'tag'->'heuristicType' as varchar) as category from jobs group by category

 * postgresql://postgres@localhost:3333/postgres
5 rows affected.


count,category
55,"""SensitiveDerefFieldCausedSynch"""
357,"""Disallow.I_CIEnter_Dialogdismiss"""
163,"""Disallow.I_CIEnter_AsyncTaskexecute"""
33,"""SensitiveDerefFieldCausedFinish"""
802,"""SensitiveDerefCallinCaused"""


In [196]:
%%sql --    # (select count(*) from jobs where status = 'new') as New_Jobs, # new jobs takes a while to run, only add if needed
select 
    (select count(*) from jobs where status = 'completed') as Completed_Jobs,
    (select count(*) from jobs where status = 'new') as New_Jobs,
    (select count(*) from jobs where status = 'paused') as Paused_Jobs,
    (select count(*) from jobs) as Total_Jobs,
    (select count(*) from jobs where status = 'acquired') as Acquired_jobs,
    (select count(id) from jobs 
       where stderr like '%Exception%no active body present for method%') as Body_Exception,
    (select count(id) from jobs where stderr like '%an implementation is missing%') as unimplemented,
    (select count(id) from jobs where stderr like '%Exception%') as Exception,
    (select count(id) from jobs where stderr like '%no libz3java in java.library.path%') as path_exception,
    (select count(id) from jobs where stderr like '%Failed to parse reflective type reference%') as Refl_exception,
    (select count(id) from jobs where status like 'failed%' and status not like 'failed: Subprocess Timeout') as worker_failure,
    (select count(id) from jobs where status like 'failed: Subprocess Timeout') as subprocess_timeout,
    (select count(id) from jobs where stdout like '%empty pred location%') as possible_cg_unsound,
    (select count(id) from jobs where stderr like '%OutOfMemory%') as memory
 

 * postgresql://postgres@localhost:3333/postgres
1 rows affected.


completed_jobs,new_jobs,paused_jobs,total_jobs,acquired_jobs,body_exception,unimplemented,exception,path_exception,refl_exception,worker_failure,subprocess_timeout,possible_cg_unsound,memory
1336,0,0,1410,0,0,0,0,0,0,35,39,0,0


In [202]:
%%sql -- get md5 hash of results table (to validate transfer)
select md5(cast((array_agg(f.* order by id)) as text)) from results f;

 * postgresql://postgres@localhost:3333/postgres
1 rows affected.


md5
5e3c67ecbc189e04252e7365bb61177a


In [201]:
#Estimate time until completion of all jobs

totTime = %sql select now() - min(started) from jobs;
completedJobs = %sql select count(*) from jobs where status = 'completed';
timeoutJobs = %sql select count(*) from jobs where status like 'failed%';
newJobs = %sql select count(*) from jobs where status = 'new';
avgJobTime = totTime[0][0] / (completedJobs[0][0] + timeoutJobs[0][0])
print("average time per job: " + str(avgJobTime))
print("est remaining time: " + str(avgJobTime * newJobs[0][0]))

 * postgresql://postgres@localhost:3333/postgres
1 rows affected.
 * postgresql://postgres@localhost:3333/postgres
1 rows affected.
 * postgresql://postgres@localhost:3333/postgres
1 rows affected.
 * postgresql://postgres@localhost:3333/postgres
1 rows affected.
average time per job: 0:01:33.808421
est remaining time: 0:00:00


In [186]:
%%sql 
select count(*),substring(status from 1 for 100) from jobs where jobs.status like 'failed%' group by substring(status from 1 for 100)


 * postgresql://postgres@localhost:3333/postgres
2 rows affected.


count,substring
18,"failed: Non-zero exit code. StdErr: Exception in thread ""main"" java.lang.OutOfMemoryError: Java hea"
17,failed: Subprocess Timeout


Print jobs and statuses

In [180]:
%%sql
select owner,count(*) as jobcount from jobs where status ='completed' group by owner order by jobcount;

 * postgresql://postgres@localhost:3333/postgres
10 rows affected.


owner,jobcount
Linux Loki 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: Loki uuid: 845ab3a8-ae10-472e-9b67-64a91be95b57,1
Linux Historia3 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: Historia3 uuid: 268fd0aa-e185-4adf-b807-476571dae6e1,1
Linux Historia3 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: Historia3 uuid: 4e682c64-2c02-491a-a254-7ae0b724b99a,1
Linux Historia3 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: Historia3 uuid: ae12c187-24ff-4019-ab85-6fa421cb87b0,1
Linux historia 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia uuid: 5fe6c467-a67c-497b-8b74-f6a20a7498f5,1
Linux historia 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia uuid: 603ef70c-2dc5-4d3c-8f31-4c4ec9d04385,1
Linux historia2-2 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia2-2 uuid: 3a2e253b-64f9-4fdc-a841-0bbc9258b880,1
Linux Loki 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: Loki uuid: 65c6ca95-8920-4eb4-ac59-b10eaf8df8f8,2
Linux historia 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia uuid: ff139dcd-e65d-41af-a1a2-65e48a18c24a,2
Linux Loki 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: Loki uuid: 793e8b81-1185-4a4c-bcf7-a24b63b54013,3


In [234]:
%%sql
select 
  (select avg(ended - started) from jobs where owner like '%Loki%' and status = 'completed') as Loki,
    (select avg(ended - started) from jobs where owner like '%Loan%' and status = 'completed') as Loan,
    (select avg(ended - started) from jobs where owner like '% historia %' and status = 'completed') as zen3,
    (select avg(ended - started) from jobs where owner not like '% historia %' and status = 'completed') as cascadelake_r

 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


loki,loan,zen3,cascadelake_r
None,None,0:01:15.268500,0:03:50.088182


In [235]:
# find exceptions
#%%sql 
#select replace(cast(stderr as varchar),'$','\$') from jobs where stderr like '%Exception%' limit 1;
outs = %sql select stderr,config from jobs where stderr like '%Exception%' and stderr not like '%OutOfMemory%' order by random() limit 1;
for out in outs:
    print(out[1])
    print(out[0])


 * postgresql://postgres@localhost:5433/postgres
0 rows affected.


In [236]:
%%sql -- look at results by tag
select cast(results.result::json->'summary' as varchar) as result, 
    cast(jobs.config::json->'tag'->'heuristicType' as varchar) as category, 
    count(*)
    from results inner join jobs on results.jobid=jobs.id where cast(jobs.config::json->'tag'->'heuristicType' as varchar)  like '%execute%'
    group by cast(results.result::json->'summary' as varchar),
    cast(jobs.config::json->'tag'->'heuristicType' as varchar)

 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


result,category,count
"""\""Unreachable\""""","""Disallow.I_CIEnter_AsyncTaskexecute""",12


In [237]:
%%sql -- results of SensitiveDerefFieldCausedSynch count
select count(*),cast(results.result::json->'summary' as varchar) from jobs inner join results on jobs.id=results.jobid 
where cast(jobs.config::json->'tag'->'heuristicType' as varchar) like '%SensitiveDerefFieldCausedSynch%' 
group by cast(results.result::json->'summary' as varchar)


 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


count,varchar
1,"""\""Unreachable\"""""


In [238]:
cfg = %sql select jobs.config from jobs inner join results on jobs.id=results.jobid where cast(jobs.config::json->'tag'->'heuristicType' as varchar) like '%SensitiveDerefFieldCausedSynch%'  and cast(results.result::json->'summary' as varchar) like '%Unreach%'

 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


In [51]:
cfg[2][0]

'{"apkPath":"${baseDir}/fdroid/org.sipdroid.sipua/6.3beta/apk/org.sipdroid.sipua_134.apk","outFolder":["${baseDirOut}/org.sipdroid.sipua"],"specSet":"{\\"specs\\":[],\\"disallow\\":[{\\"univQuant\\":[{\\"$type\\":\\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\\",\\"n\\":\\"a\\"},{\\"$type\\":\\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\\",\\"n\\":\\"f\\"}],\\"existQuant\\":[],\\"pred\\":{\\"$type\\":\\"edu.colorado.plv.bounder.lifestate.LifeState.Or\\",\\"l1\\":{\\"$type\\":\\"edu.colorado.plv.bounder.lifestate.LifeState.NS\\",\\"i1\\":{\\"$type\\":\\"edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\\",\\"mt\\":{\\"$type\\":\\"edu.colorado.plv.bounder.ir.CBExit\\"},\\"signatures\\":{\\"$type\\":\\"edu.colorado.plv.bounder.lifestate.LifeState.SubClassMatcher\\",\\"baseSubtypeOf\\":[\\"android.app.Fragment\\",\\"androidx.fragment.app.Fragment\\",\\"android.support.v4.app.Fragment\\"],\\"signatureMatcher\\":\\"void onDestroy\\\\\\\\(\\\\\\\\)\\",\\"

In [239]:
%%sql
select cast(results.result::json->'summary' as varchar) as summary, cast(jobs.config::json->'apkPath' as varchar) as benchmark,count(*) from results inner join jobs on jobs.id=results.jobid group by summary,benchmark order by benchmark,summary desc

 * postgresql://postgres@localhost:5433/postgres
11 rows affected.


summary,benchmark,count
"""\""Unreachable\""""","""${baseDir}/fdroid/com.android.keepass/2.6.8/apk/com.android.keepass_215.apk""",1
"""\""Unreachable\""""","""${baseDir}/fdroid/com.gpl.rpg.AndorsTrail/0.8.3/apk/com.gpl.rpg.AndorsTrail_67.apk""",1
"""\""Unreachable\""""","""${baseDir}/fdroid/com.owncloud.android/3.0.1/apk/com.owncloud.android_30000001.apk""",1
"""\""Unreachable\""""","""${baseDir}/fdroid/com.shatteredpixel.shatteredpixeldungeon/1.4.3/apk/com.shatteredpixel.shatteredpixeldungeon_668.apk""",1
"""\""Unreachable\""""","""${baseDir}/fdroid/de.blinkt.openvpn/0.7.43/apk/de.blinkt.openvpn_198.apk""",1
"""\""Unreachable\""""","""${baseDir}/fdroid/de.markusfisch.android.wavelines/1.13.2/apk/de.markusfisch.android.wavelines_22.apk""",1
"""\""Unreachable\""""","""${baseDir}/fdroid/dev.ukanth.ufirewall/3.5.3/apk/dev.ukanth.ufirewall_20220615.apk""",6
"""\""Witnessed\""""","""${baseDir}/fdroid/net.wigle.wigleandroid/2.63/apk/net.wigle.wigleandroid_263.apk""",1
"""\""Witnessed\""""","""${baseDir}/fdroid/org.liberty.android.fantastischmemo/10.11.7/apk/org.liberty.android.fantastischmemo_237.apk""",1
"""\""Unreachable\""""","""${baseDir}/fdroid/org.liberty.android.fantastischmemo/10.11.7/apk/org.liberty.android.fantastischmemo_237.apk""",2


Get failing apks

In [240]:
%%sql
select cast(config::json->'apkPath' as text) from jobs 
        where stderr like '%Exception%' group by cast(config::json->'apkPath' as text) ;

 * postgresql://postgres@localhost:5433/postgres
0 rows affected.


text


In [241]:
%%sql
select cast(config::json->'apkPath' as text) as app,count(*) as count from jobs group by app order by count desc;

 * postgresql://postgres@localhost:5433/postgres
49 rows affected.


app,count
"""${baseDir}/fdroid/net.osmand.plus/4.3.8/apk/net.osmand.plus_4308.apk""",285
"""${baseDir}/fdroid/org.sufficientlysecure.keychain/5.8.2/apk/org.sufficientlysecure.keychain_58902.apk""",143
"""${baseDir}/fdroid/org.mariotaku.twidere/4.1.8/apk/org.mariotaku.twidere_517.apk""",92
"""${baseDir}/fdroid/net.wigle.wigleandroid/2.63/apk/net.wigle.wigleandroid_263.apk""",88
"""${baseDir}/fdroid/dev.ukanth.ufirewall/3.5.3/apk/dev.ukanth.ufirewall_20220615.apk""",73
"""${baseDir}/fdroid/com.seafile.seadroid2/2.2.48/apk/com.seafile.seadroid2_123.apk""",63
"""${baseDir}/fdroid/de.blau.android/18.1.1.0/apk/de.blau.android_2303.apk""",59
"""${baseDir}/fdroid/org.liberty.android.fantastischmemo/10.11.7/apk/org.liberty.android.fantastischmemo_237.apk""",56
"""${baseDir}/fdroid/de.blinkt.openvpn/0.7.43/apk/de.blinkt.openvpn_198.apk""",44
"""${baseDir}/fdroid/com.ichi2.anki/2.15.6/apk/com.ichi2.anki_21506300.apk""",41


In [242]:
%%sql 
select cast(result::json->'summary' as varchar) as summary, cast(result::json->'maxPathCh' as varchar) as ch,count(*),avg(querytime),max(querytime) from results group by summary,ch order by summary,ch;

 * postgresql://postgres@localhost:5433/postgres
2 rows affected.


summary,ch,count,avg,max
"""\""Unreachable\""""","""\""UnknownCharacterization\""""",15,1.00000000000000000000,7
"""\""Witnessed\""""","""\""UnknownCharacterization\""""",2,2.0000000000000000,3


In [243]:
%%sql 
select count(*), cast(results.result::json->'summary' as varchar),cast(jobs.config::json->'tag' as varchar) as tag 
from jobs inner join results on jobs.id=results.jobid group by cast(jobs.config::json->'tag' as varchar),cast(results.result::json->'summary' as varchar)
order by  cast(jobs.config::json->'tag' as varchar)

 * postgresql://postgres@localhost:5433/postgres
4 rows affected.


count,varchar,tag
12,"""\""Unreachable\""""","{""heuristicType"":""Disallow.I_CIEnter_AsyncTaskexecute""}"
2,"""\""Unreachable\""""","{""heuristicType"":""SensitiveDerefCallinCaused""}"
2,"""\""Witnessed\""""","{""heuristicType"":""SensitiveDerefCallinCaused""}"
1,"""\""Unreachable\""""","{""heuristicType"":""SensitiveDerefFieldCausedSynch""}"


In [244]:
%%sql 
select regexp_substr(cast(jobs.config::json->'apkPath' as varchar), '[a-zA-Z]*\.[a-zA-Z]*') as benchmark, 
    regexp_substr(cast(results.qry::json->'s' as varchar), 'Dialog_dismiss|execute') as isDismiss, 
    cast(results.result::json->'summary' as varchar) as summary, count(*)
    from results inner join jobs on results.jobid=jobs.id 
    group by cast(jobs.config::json->'apkPath' as varchar),cast(results.result::json->'summary' as varchar),
    regexp_substr(cast(results.qry::json->'s' as varchar), 'Dialog_dismiss|execute')


 * postgresql://postgres@localhost:5433/postgres
11 rows affected.


benchmark,isdismiss,summary,count
com.android,execute,"""\""Unreachable\""""",1
com.gpl,execute,"""\""Unreachable\""""",1
com.owncloud,execute,"""\""Unreachable\""""",1
com.shatteredpixel,None,"""\""Unreachable\""""",1
de.blinkt,None,"""\""Unreachable\""""",1
de.markusfisch,execute,"""\""Unreachable\""""",1
dev.ukanth,execute,"""\""Unreachable\""""",6
net.wigle,None,"""\""Witnessed\""""",1
org.liberty,execute,"""\""Unreachable\""""",2
org.liberty,None,"""\""Witnessed\""""",1


In [248]:
%%sql
select cast(result::json->'summary' as varchar) as summary, count(*) from results group by cast(result::json->'summary' as varchar)

 * postgresql://postgres@localhost:5433/postgres
2 rows affected.


summary,count
"""\""Unreachable\""""",15
"""\""Witnessed\""""",2


In [257]:
%%sql
select count(*),status from jobs group by status;

 * postgresql://postgres@localhost:5433/postgres
5 rows affected.


count,status
5,failed: java.util.concurrent.TimeoutException: Future timed out after [300 seconds]java.util.concurrent.TimeoutException: Future timed out after [300 seconds] at scala.concurrent.impl.Promise$DefaultPromise.tryAwait0(Promise.scala:248) at scala.concurrent.impl.Promise$DefaultPromise.result(Promise.scala:261) at scala.concurrent.Await$.$anonfun$result$1(package.scala:201) at scala.concurrent.BlockContext$DefaultBlockContext$.blockOn(BlockContext.scala:62) at scala.concurrent.Await$.result(package.scala:124) at edu.colorado.plv.bounder.ExperimentsDb.downloadApk(Driver.scala:1239) at edu.colorado.plv.bounder.ExperimentsDb.getInputs(Driver.scala:1287) at edu.colorado.plv.bounder.ExperimentsDb.$anonfun$processJob$1(Driver.scala:865) at edu.colorado.plv.bounder.ExperimentsDb.processJob(Driver.scala:934) at edu.colorado.plv.bounder.ExperimentsDb.loop(Driver.scala:838) at edu.colorado.plv.bounder.Driver$.expLoop(Driver.scala:281) at edu.colorado.plv.bounder.Driver$.runAction(Driver.scala:343) at edu.colorado.plv.bounder.Driver$.main(Driver.scala:268) at edu.colorado.plv.bounder.Driver.main(Driver.scala)
13,completed
37,acquired
1348,new
7,failed: java.util.concurrent.TimeoutException: Future timed out after [300 seconds]java.util.concurrent.TimeoutException: Future timed out after [300 seconds] at scala.concurrent.impl.Promise$DefaultPromise.tryAwait0(Promise.scala:248) at scala.concurrent.impl.Promise$DefaultPromise.result(Promise.scala:261) at scala.concurrent.Await$.$anonfun$result$1(package.scala:201) at scala.concurrent.BlockContext$DefaultBlockContext$.blockOn(BlockContext.scala:62) at scala.concurrent.Await$.result(package.scala:124) at edu.colorado.plv.bounder.ExperimentsDb.downloadApk(Driver.scala:1239) at edu.colorado.plv.bounder.ExperimentsDb.$anonfun$processJob$1(Driver.scala:857) at edu.colorado.plv.bounder.ExperimentsDb.processJob(Driver.scala:934) at edu.colorado.plv.bounder.ExperimentsDb.loop(Driver.scala:838) at edu.colorado.plv.bounder.Driver$.expLoop(Driver.scala:281) at edu.colorado.plv.bounder.Driver$.runAction(Driver.scala:343) at edu.colorado.plv.bounder.Driver$.main(Driver.scala:268) at edu.colorado.plv.bounder.Driver.main(Driver.scala)


Check result consistency
========================

In [252]:
%%sql -- get workers with acquired jobs -- check for abandoned jobs
select owner,count(*) from jobs where status='acquired' group by owner;

 * postgresql://postgres@localhost:5433/postgres
37 rows affected.


owner,count
Linux historia2-2 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia2-2 uuid: 050e1c59-e514-4604-8d20-0d7105ca1c20,1
Linux historia2-2 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia2-2 uuid: 38afe6a5-a557-4c6c-a371-7f43c834eb9d,1
Linux historia2-2 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia2-2 uuid: 57304e84-01fb-4dae-9997-3b3153d1fb04,1
Linux historia2-2 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia2-2 uuid: 73aeccb9-eb8e-4774-8acf-6652292ba835,1
Linux historia2-2 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia2-2 uuid: 75b844be-c068-499b-bd97-4e3caaaa38df,1
Linux historia2-2 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia2-2 uuid: 9acaa80e-588a-4b74-9e07-047b736f694e,1
Linux historia2-2 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia2-2 uuid: c0cafe56-ee4e-49f0-ae84-9185d8fe61eb,1
Linux historia2 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia2 uuid: 208957a3-9467-49a0-9b4e-f5b9980955d2,1
Linux historia2 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia2 uuid: 3139a2b4-d06b-45f9-81ba-a9e6525b6081,1
Linux historia2 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia2 uuid: 5ab70004-e9bf-4afe-9fd3-070051fcb62f,1


In [253]:
%%sql -- Find completed jobs where no results exist and no error occurred
select jobs.id from jobs where jobs.id not in (select results.jobid from results) and jobs.status = 'completed' and jobs.stderr not like '%Exception%';

 * postgresql://postgres@localhost:5433/postgres
0 rows affected.


id


In [254]:
%%sql -- Find new/paused jobs with results
select jobs.id from jobs where jobs.id in (select results.jobid from results) and (jobs.status = 'paused' or jobs.status = 'new');

 * postgresql://postgres@localhost:5433/postgres
0 rows affected.


id


In [255]:
%%sql -- check that each location/qry pair only has one result (should detect duplicat runs)
select count(*),qry,loc as num from results group by qry,loc order by count(*) desc limit 1;

 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


count,qry,num
1,"{""t"":""DisallowedCallin"",""className"":""dev.ukanth.ufirewall.activity.LogDetailActivity"",""methodName"":""boolean onContextItemSelected(android.view.MenuItem)"",""s"":""{\""univQuant\"":[{\""$type\"":\""edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\"",\""n\"":\""t\""}],\""existQuant\"":[],\""pred\"":{\""$type\"":\""edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\"",\""mt\"":{\""$type\"":\""edu.colorado.plv.bounder.ir.CIExit\""},\""signatures\"":{\""$type\"":\""edu.colorado.plv.bounder.lifestate.LifeState.SubClassMatcher\"",\""baseSubtypeOf\"":[\""android.os.AsyncTask\""],\""signatureMatcher\"":\"".*AsyncTask execute\\\\(.*\\\\)\"",\""ident\"":\""AsyncTask_execute\""},\""lsVars\"":[{\""$type\"":\""edu.colorado.plv.bounder.symbolicexecutor.state.TopVal\""},{\""$type\"":\""edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\"",\""n\"":\""t\""}]},\""target\"":{\""$type\"":\""edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\"",\""mt\"":{\""$type\"":\""edu.colorado.plv.bounder.ir.CIEnter\""},\""signatures\"":{\""$type\"":\""edu.colorado.plv.bounder.lifestate.LifeState.SubClassMatcher\"",\""baseSubtypeOf\"":[\""android.os.AsyncTask\""],\""signatureMatcher\"":\"".*AsyncTask execute\\\\(.*\\\\)\"",\""ident\"":\""AsyncTask_execute\""},\""lsVars\"":[{\""$type\"":\""edu.colorado.plv.bounder.symbolicexecutor.state.TopVal\""},{\""$type\"":\""edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\"",\""n\"":\""t\""}]}}""}","{""$type"":""edu.colorado.plv.bounder.ir.AppLoc"",""method"":[""boolean onContextItemSelected(android.view.MenuItem)"",""dev.ukanth.ufirewall.activity.LogDetailActivity"",[""dev.ukanth.ufirewall.activity.LogDetailActivity"",""android.view.MenuItem""],[""[{\""name\"":\""@this\"",\""localType\"":\""dev.ukanth.ufirewall.activity.LogDetailActivity\""}]"",""[{\""name\"":\""@parameter0\"",\""localType\"":\""android.view.MenuItem\""}]""]],""line"":{""id"":213,""desc"":""SerializedIRLineLoc(213,line: 213 virtualinvoke $r2.<dev.ukanth.ufirewall.util.LogNetUtil$NetTask: android.os.AsyncTask execute(java.lang.Object[])>($r3),430910893)"",""ident"":1835841364},""isPre"":true}"


Triage witness results
----------------------

In [ ]:
witnessesq = %sql \
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results \
                where cast(result::json->'summary' as varchar) like '%Witnessed%' order by id asc;

witnessesdf = witnessesq.DataFrame()
witnessesdf.to_csv('triageWit.csv')

In [ ]:
witnessesq = %sql \
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results \
                where cast(result::json->'summary' as varchar) like '%Itimeout%' and cast(result::json->'maxPathCh' as varchar) like '%MultiCallback%' order by id asc;

witnessesdf = witnessesq.DataFrame()
witnessesdf.to_csv('triageWitMultiTimeout.csv')

In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz, 
        replace(cast(loc::json->'line'->'str' as varchar),'$','\$') from results where id = 40;

Triage proven results
--------------------------

In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results 
                where cast(result::json->'summary' as varchar) like '%Proven%' order by random() limit 10;

Triage unreachable results
--------------------------

In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz 
            from results where cast(result::json->'summary' as varchar) like '%Unreachable%' 
            order by random() limit 15;

In [ ]:
%%sql
select id,jobid,resultdata,loc::json->'method'->>1 as clazz, 
        replace(cast(loc::json->'method'->>0 as varchar),'$','\$') as mname, 
        replace(cast(loc::json->'line' as varchar),'$','\$') as location 
        from results where id = 15037;

In [ ]:
%%sql
select id,result,replace(cast(loc::json->'line' as varchar),'$','\$') as location 
    from results where loc like '%244 $r3%';

In [ ]:
%%sql
select replace(cast(loc as varchar),'$','\$') from results where id = 1494;

Runtime Stats
=============

In [ ]:
queryTime = %sql select querytime/60/60 from results;
tplt = queryTime.DataFrame().plot.hist(bins=16)
tplt.set_yscale('log')
# tplt.set_xscale('log')
tplt.set_xlabel('time (hrs)')
# split this into proven and witnessed

In [ ]:
%%sql
select 

In [ ]:
cfg = %sql select jobs.config from results join jobs on jobs.id = results.jobid where querytime > 6*60*60 limit 1;
if len(cfg) > 0:
    print(cfg[0][0])

In [ ]:
%%sql -- which apps cause runtimes over 2 hours?
select jobs.id, results.querytime, jobs.config::json->'apkPath' from results inner join jobs on jobs.id=results.jobid where results.querytime > 7200 order by results.querytime desc limit 200;

In [ ]:
# Job runtimes
runtimes = %sql select ended - started as time from jobs where ended is not NULL;
runtimesDF = runtimes.DataFrame()
runtimesDF['time'] = runtimesDF['time'].apply(lambda x: x.total_seconds() / (60))
tplt = runtimesDF.plot.hist(bins=32)
tplt.set_yscale('log')
tplt.set_xlabel('time (min)')

In [ ]:
# Plot job time trend
rtvt = %sql select started, ended - started as time from jobs where ended is not NULL;
rtvtDF = rtvt.DataFrame()
rtvtDF.sort_values('started', ascending=True)
fig, ax = plt.subplots()
ax.plot_date(rtvtDF['started'], rtvtDF['time'], marker='*', linestyle='')

fig.autofmt_xdate()
plt.show()


Total job time

In [ ]:
%%sql
select sum(ended-started) from jobs where ended is not NULL;

In [ ]:
%%sql
select avg(ended-started) from jobs where ended is not NULL;

In [ ]:
%%sql
select avg(querytime) from results;

In [ ]:
%%sql
select now() - min(started) from jobs;

Size of postgres data
=====================

In [ ]:
%%sql
WITH RECURSIVE pg_inherit(inhrelid, inhparent) AS
    (select inhrelid, inhparent
    FROM pg_inherits
    UNION
    SELECT child.inhrelid, parent.inhparent
    FROM pg_inherit child, pg_inherits parent
    WHERE child.inhparent = parent.inhrelid),
pg_inherit_short AS (SELECT * FROM pg_inherit WHERE inhparent NOT IN (SELECT inhrelid FROM pg_inherit))
SELECT table_schema
    , TABLE_NAME
    , row_estimate
    , pg_size_pretty(total_bytes) AS total
    , pg_size_pretty(index_bytes) AS INDEX
    , pg_size_pretty(toast_bytes) AS toast
    , pg_size_pretty(table_bytes) AS TABLE
  FROM (
    SELECT *, total_bytes-index_bytes-COALESCE(toast_bytes,0) AS table_bytes
    FROM (
         SELECT c.oid
              , nspname AS table_schema
              , relname AS TABLE_NAME
              , SUM(c.reltuples) OVER (partition BY parent) AS row_estimate
              , SUM(pg_total_relation_size(c.oid)) OVER (partition BY parent) AS total_bytes
              , SUM(pg_indexes_size(c.oid)) OVER (partition BY parent) AS index_bytes
              , SUM(pg_total_relation_size(reltoastrelid)) OVER (partition BY parent) AS toast_bytes
              , parent
          FROM (
                SELECT pg_class.oid
                    , reltuples
                    , relname
                    , relnamespace
                    , pg_class.reltoastrelid
                    , COALESCE(inhparent, pg_class.oid) parent
                FROM pg_class
                    LEFT JOIN pg_inherit_short ON inhrelid = oid
                WHERE relkind IN ('r', 'p')
             ) c
             LEFT JOIN pg_namespace n ON n.oid = c.relnamespace
  ) a
  WHERE oid = parent
) a
ORDER BY total_bytes DESC limit 5;

Inspection of exceptions
========================

In [ ]:
%%sql
select id,config::json->'apkPath',replace(stderr,'$','\$') from jobs where stderr like '%Exception%';

In [ ]:
#count of total results
%sql select count(*) as Total_locations_computed from results